In [1]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
import cv2 
import random
import numpy as np
import time

C:\Users\yingy\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\yingy\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.4SP5SUA7CBGXUEOC35YP2ASOICYYEQZZ.gfortran-win_amd64.dll
C:\Users\yingy\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
os.getcwd()

'c:\\Users\\yingy\\OneDrive\\Escritorio\\FaceMask_Model'

# Setup Paths

In [5]:
MODEL_PATH = 'models/ckpt-16'
LABEL_PATH = 'models/annotations/label_map.pbtxt'
CONFIG_PATH = 'models/pipeline.config'

In [6]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(MODEL_PATH).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [7]:
category_index = label_map_util.create_category_index_from_labelmap(LABEL_PATH)

In [8]:
category_index

{1: {'id': 1, 'name': 'Mask'},
 2: {'id': 2, 'name': 'NoMask'},
 3: {'id': 3, 'name': 'NotProp'}}

In [9]:
font = cv2.FONT_HERSHEY_SIMPLEX
upperLeftCornerOfText = (10, 30)
upperLeftCornerOfText2 = (10, 60)
bottomLeftCornerOfText = (10, 450)
fontScale = 1
fontColor = (0,100,0)
lineType = 2
label_id_offset = 1
try:
    
    j = 10
    i = 0
    cap = cv2.VideoCapture(0)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    
    while True:
        message = f'Press "s" to start'
        ret, frame = cap.read()
        image_np = np.array(frame)

        input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
        detections = detect_fn(input_tensor)

        num_detections = int(detections.pop('num_detections'))
        detections = {key: value[0, :num_detections].numpy()
                      for key, value in detections.items()}
        detections['num_detections'] = num_detections

        # detection_classes should be ints.
        detections['detection_classes'] = detections['detection_classes'].astype(np.int64)


        image_np_with_detections = image_np.copy()

        viz_utils.visualize_boxes_and_labels_on_image_array(
                    image_np_with_detections,
                    detections['detection_boxes'],
                    detections['detection_classes']+label_id_offset,
                    detections['detection_scores'],
                    category_index,
                    use_normalized_coordinates=True,
                    max_boxes_to_draw=1,
                    min_score_thresh=.5,
                    agnostic_mode=False)
        cv_toshow = cv2.resize(image_np_with_detections, (800, 600))
        cv2.putText(cv_toshow, message, 
            upperLeftCornerOfText, 
            font,
            fontScale,
            fontColor,
            lineType)
        cv2.imshow('object detection',  cv_toshow)
        k = cv2.waitKey(1) & 0xFF
        if k == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            break
        elif k == ord('s'):
            time_start = time.time()
            while True:
                time_current = time.time() - time_start
                countdown = round(3.4 - time_current)
                message = f'Show your hand in {countdown}'
                ret, frame = cap.read()
                image_np = np.array(frame)

                input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
                detections = detect_fn(input_tensor)

                num_detections = int(detections.pop('num_detections'))
                detections = {key: value[0, :num_detections].numpy()
                              for key, value in detections.items()}
                detections['num_detections'] = num_detections

                # detection_classes should be ints.
                detections['detection_classes'] = detections['detection_classes'].astype(np.int64)


                image_np_with_detections = image_np.copy()

                viz_utils.visualize_boxes_and_labels_on_image_array(
                            image_np_with_detections,
                            detections['detection_boxes'],
                            detections['detection_classes']+label_id_offset,
                            detections['detection_scores'],
                            category_index,
                            use_normalized_coordinates=True,
                            max_boxes_to_draw=1,
                            min_score_thresh=.4,
                            agnostic_mode=False)
                cv_toshow = cv2.resize(image_np_with_detections, (800, 600))
                cv2.putText(cv_toshow, message, 
                    upperLeftCornerOfText, 
                    font,
                    fontScale,
                    fontColor,
                    lineType)
                cv2.imshow('object detection',  cv_toshow)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    cap.release()
                    cv2.destroyAllWindows()
                    break
                if countdown <= 0:
                    time_1 = time.time()
                    decision = category_index[random.choice(list(category_index.keys()))]['name']
                    while True:
                        time_within = time.time() - time_1
                        ret, frame = cap.read()
                        image_np = np.array(frame)
                        input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
                        detections = detect_fn(input_tensor)
                        num_detections = int(detections.pop('num_detections'))
                        detections = {key: value[0, :num_detections].numpy()
                              for key, value in detections.items()}
                        detections['num_detections'] = num_detections
                        detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
                        image_np_with_detections = image_np.copy()
                        viz_utils.visualize_boxes_and_labels_on_image_array(
                            image_np_with_detections,
                            detections['detection_boxes'],
                            detections['detection_classes']+label_id_offset,
                            detections['detection_scores'],
                            category_index,
                            use_normalized_coordinates=True,
                            max_boxes_to_draw=1,
                            min_score_thresh=.4,
                            agnostic_mode=False)
                        if cv2.waitKey(1) & 0xFF == ord('q'):
                            cap.release()
                            cv2.destroyAllWindows()
                            break
                        cv_toshow = cv2.resize(image_np_with_detections, (800, 600))
                        cv2.putText(cv_toshow, f'I am choosing: {decision}.', 
                            upperLeftCornerOfText, 
                            font,
                            fontScale,
                            fontColor,
                            lineType)
                        cv2.imshow('object detection',  cv_toshow)
                        if time_within > 0.5:
                            time_2 = time.time()
                            user_decision = category_index[(detections['detection_classes']+label_id_offset)[0]]['name']
                            while True:
                                time_within = time.time() - time_2
                                ret, frame = cap.read()
                                image_np = np.array(frame)
                                image_toshow = cv2.resize(image_np, (800, 600))
                                cv2.putText(image_toshow, f'I am choosing: {decision}.', 
                                    upperLeftCornerOfText, 
                                    font,
                                    fontScale,
                                    fontColor,
                                    lineType)
                                cv2.putText(image_toshow, f'You chose {user_decision}.', 
                                    upperLeftCornerOfText2, 
                                    font,
                                    fontScale,
                                    fontColor,
                                    lineType)
                                cv2.imshow('object detection', image_toshow)
                                if time_within > 3:
                                    break
                                if cv2.waitKey(1) & 0xFF == ord('q'):
                                    cap.release()
                                    cv2.destroyAllWindows()
                                    break
                            break

                    time_start = time.time()
        
        
except:
    cap.release()

In [12]:
detections.keys()


dict_keys(['detection_boxes', 'detection_scores', 'detection_classes', 'raw_detection_boxes', 'raw_detection_scores', 'detection_multiclass_scores', 'detection_anchor_indices', 'num_detections'])

In [21]:
detections['detection_scores']

array([0.7493779 , 0.0757446 , 0.07273477, 0.06574634, 0.05184636,
       0.05139107, 0.05022532, 0.04096368, 0.038109  , 0.03804481,
       0.03705531, 0.03486669, 0.03170073, 0.02871361, 0.02862534,
       0.02852809, 0.02644783, 0.0263795 , 0.02613997, 0.02584374,
       0.02524531, 0.0249882 , 0.02454662, 0.02410406, 0.02387947,
       0.02359322, 0.02334237, 0.02327237, 0.02271512, 0.02172542,
       0.02162555, 0.02129379, 0.0212481 , 0.02103397, 0.02063909,
       0.0206289 , 0.02044153, 0.01955476, 0.01935366, 0.01926675,
       0.01924458, 0.01847324, 0.01831242, 0.01763806, 0.01749778,
       0.01746088, 0.01738372, 0.01704475, 0.01686808, 0.0167166 ,
       0.01634231, 0.01608232, 0.01605856, 0.01592144, 0.01579261,
       0.0157325 , 0.01562235, 0.0155465 , 0.01553187, 0.0154134 ,
       0.01524556, 0.01522401, 0.01520467, 0.01519942, 0.01511937,
       0.01498556, 0.01495937, 0.0149084 , 0.01485813, 0.01473564,
       0.01464534, 0.0146409 , 0.01440626, 0.01433784, 0.01426

In [22]:
detections['detection_classes']

array([0, 0, 0, 2, 0, 0, 2, 0, 2, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2,
       0, 2, 2, 1, 1, 2, 0, 2, 1, 2, 0, 2, 2, 2, 2, 0, 2, 2, 1, 2, 2, 2,
       1, 0, 2, 0, 0, 0, 0, 1, 0, 1, 2, 2, 2, 0, 0, 0, 0, 2, 0, 0, 2, 2,
       2, 0, 2, 1, 2, 2, 0, 2, 0, 0, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 0, 0,
       0, 2, 2, 0, 2, 0, 2, 2, 0, 0, 2, 2], dtype=int64)